In [1]:
import sys
sys.executable

'/Users/arkin/cs229_materials/closed-question-detection/venv/bin/python'

In [2]:
import pandas as pd
import os
FOLDER_PATH="../so_dataset"
from utils import body_strip_tags, just_text, filter_sentence, get_tag_list, remove_filpaths

In [3]:
df_total_features = pd.read_csv(os.path.join(FOLDER_PATH,"so_questions_labelled.csv"))

In [4]:
index = 0
df_total_features['Qid'].iloc[index],df_total_features['Label'].iloc[index]

(64303805, 3)

In [5]:
df_total_features['Title'].iloc[index], df_total_features['Tags'].iloc[index]

('How to convert a Python list of dictionary with same key value into dictionary with list of values',
 '<python><python-3.x><dictionary>')

In [6]:
df_total_features['title'] = df_total_features['Title'].apply(lambda x: ' '.join(filter_sentence(x)).lower())

In [7]:
df_total_features['tag_list'] = df_total_features['Tags'].apply(lambda x: get_tag_list(x))

In [8]:
df_total_features['Body'].iloc[index]

"<p>I have a list of dictionary like this:</p>\n<pre><code>mylist = [\n    {'survival': 124, 'organ': 'Stomach'}\n    {'survival': 42, 'organ': 'Stomach'}\n    ]\n</code></pre>\n<p>I want to change this into a dictionary with same keys to have list of value like so:</p>\n<pre><code>myDict = {\n  &quot;survival&quot;:[124,42],\n  &quot;organ&quot;:[&quot;Stomach&quot;,&quot;Stomach&quot;]\n}\n</code></pre>\n"

In [9]:
body_strip_tags(just_text(df_total_features['Body'].iloc[index]))

'I have a list of dictionary like this:I want to change this into a dictionary with same keys to have list of value like so:'

In [10]:
df_total_features['body'] = df_total_features['Body'].apply(just_text).apply(body_strip_tags)

In [11]:
df_total_features['body'] = df_total_features['body'].apply(remove_filpaths)

/Users/arkin/cs229_materials/closed-question-detection/data_extraction_cleaning/utils.py:34: FutureWarning: Possible nested set at position 1
  return re.sub(r'[[\w\-\:]*\/[\w\-\:]*\/[\w\-\:]*]*|[[\w\-\:\.]*\\[\w\-\:]*\\[\w\-\:]*]*', "", x)
/Users/arkin/cs229_materials/closed-question-detection/data_extraction_cleaning/utils.py:34: FutureWarning: Possible nested set at position 36
  return re.sub(r'[[\w\-\:]*\/[\w\-\:]*\/[\w\-\:]*]*|[[\w\-\:\.]*\\[\w\-\:]*\\[\w\-\:]*]*', "", x)


In [16]:
from spacy.lang.en import English
nlp = English()
tokenizer = nlp.Defaults.create_tokenizer(nlp)

def tokenize(text):
    tokens = []
    doc = tokenizer(text)
    for token in doc:
        if ('d' in token.shape_):
            # token contains digit
            continue
        else:
            tokens.append(token.text)
    return '|'.join(tokens).lower()

In [17]:
df_cleaned = df_total_features.dropna( how='any')

In [18]:
df_cleaned['title'] = df_cleaned['title'].apply(tokenize)
df_cleaned['body'] = df_cleaned['body'].apply(tokenize)

In [19]:
df_cleaned.to_csv(os.path.join(FOLDER_PATH, "so_questions_cleaned.csv"), index=False)

In [21]:
df_cleaned['body']

0       i|have|a|list|of|dictionary|like|this|:|i|want...
1       i|'m|porting|an|elixir|library|to|elm|that|use...
2       trying|to|filter|boolean|field|,|but|it|brings...
3       consider|the|following|code|:|i|create|a|funct...
4       not|able|to|get|value|of|gridview|cells|.|i|am...
                              ...                        
6143    i|want|to|show|an|section|when|the|checkbox|is...
6144    [|cpu|use|pytorch|to|train|dnn|,|but|i|find|th...
6145    i|was|install|kubernetes|by|kubespray|,|my|kub...
6146    what|is|wrong|in|code|?|why|the|redirection|to...
6147    i|have|a|data|set|with|scores|and|categories|i...
Name: body, Length: 6148, dtype: object

In [25]:
df_cleaned['Label'].value_counts()

0    2000
1    1848
2    1374
3     816
4     110
Name: Label, dtype: int64